<a href="https://colab.research.google.com/github/enVives/TFG/blob/main/EvaluacioArquitectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import cv2
import random
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import wandb
import pylab as pl


from glob import glob
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import train_test_split
from torchvision import transforms,models
from torch import nn
from torch.utils.data import Dataset
from google.colab import files
from IPython.display import clear_output,display
from google.colab.patches import cv2_imshow
from PIL import Image
from skimage import io

WEIGHTSANDBIASES = False
DOWNLOAD = True
BINARY = True

TRAINING = 0.64
VALIDATION = 0.16
TESTING = 0.20
SIZE = 224


if DOWNLOAD:
  !rm -rf /content/sample_data/*

  !kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

  !unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

#471be466c8949671a46c67e7aad0d5a0ac8c9dad

if WEIGHTSANDBIASES:
  wandb.login()

torch.cuda.default_stream(torch.device('cuda'))

!rm -rf /content/sample_data/*

!kaggle datasets download -d "kmader/skin-cancer-mnist-ham10000"

!unzip -o skin-cancer-mnist-ham10000.zip -d /content/sample_data/

In [ ]:
metadates = pd.read_csv('/content/sample_data/HAM10000_metadata.csv')
metadates = metadates.sort_values(by='image_id')
metadates.head()
print(metadates['dx'].value_counts())
print()
print(metadates['dx'].value_counts() / sum(metadates['dx'].value_counts()))

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

dx
nv       0.669496
mel      0.111133
bkl      0.109735
bcc      0.051323
akiec    0.032651
vasc     0.014179
df       0.011483
Name: count, dtype: float64


nv: melanocytic nevi

vasc: vascular lesions

mel:melanoma

df: dermatofibroma

bkl: benign keratosis-like lesions

bcc: basal cell carcinoma

akiec: Actinic keratoses and intraepithelial carcinoma / Bowen's disease

In [ ]:
class Formes(Dataset):
  #Classe on gestionarem les imatges dels fitxers
  dict_illnesses = {0 : 'nv', 1 : 'mel', 2 : 'bkl', 3 : 'bcc', 4 : 'akiec', 5 : 'vasc', 6 : 'df'}

  def __init__(self, images, labels, transform):
        super().__init__()
        self.paths = images
        self.labels = labels
        self.len = len(self.paths)
        self.transform = transform
        #Per defecte pens que el color pot extreure característiques importants, per tant en primer lloc
        #entrenarem les imatges de color
        self.greyscale = False

  def __len__(self):
        return self.len

  def __addlabel__(self,label):
    self.labels = np.append(self.labels,label)

  def __addPath__(self,path):
    self.paths = np.append(self.paths,path)

  def __getdist__(self):
    return pd.Series(self.labels).value_counts()

  def __getlabels__(self):
    classes = [[],[],[],[],[],[],[]]
    for i in range(len(self.labels)):
      classes[self.labels[i]].append(i) #afegim l'index
    return classes

  def __redistribute__(self,percentages):
    #percentages: [15,15,10,10,5,5] percentatges que volem pujar de la resta de classes llevat de nv
    threshold = 0.005  #percentatge de marge que deixam a la redistribució
    Ntarget = self.len
    classes = self.__getlabels__() #indexos de cada clase
    afegir = np.array([0,0,0,0,0,0],dtype=np.int64) # de nv mai haurem d'afegir

    nmel = len(classes[1]) #nombre inicial de cada clase
    nbkl = len(classes[2])
    nbcc = len(classes[3])
    nakiec = len(classes[4])
    nvasc = len(classes[5])
    ndf = len(classes[6])

    while True:

      suma_actual = afegir.sum()

      operacio = percentages[0]*Ntarget - nmel
      afegir[0] +=  operacio if operacio > 0 else 0
      nmel += operacio if operacio > 0 else 0

      operacio = percentages[1]*Ntarget - nbkl
      afegir[1] += operacio if operacio > 0 else 0
      nbkl += operacio if operacio > 0 else 0

      operacio = percentages[2]*Ntarget - nbcc
      afegir[2] += operacio if operacio > 0 else 0
      nbcc += operacio if operacio > 0 else 0

      operacio = percentages[3]*Ntarget - nakiec
      afegir[3] += operacio if operacio > 0 else 0
      nakiec += operacio if operacio > 0 else 0

      operacio = percentages[4]*Ntarget - nvasc
      afegir[4] += operacio if operacio > 0 else 0
      nvasc += operacio if operacio > 0 else 0

      operacio = percentages[5]*Ntarget - ndf
      afegir[5] += operacio if operacio > 0 else 0
      ndf += operacio if operacio > 0 else 0

      if (afegir.sum()-suma_actual) < Ntarget*threshold:
        break

      Ntarget += (afegir.sum()-suma_actual)

    #Quedaria afegir a les imatges les còpies
    for i in range(len(afegir)):
      for j in range(afegir[i]):

          self.__addPath__(self.paths[classes[i+1][random.randint(0, len(classes[i+1]) - 1)]])
          self.__addlabel__(i+1)

    self.len = len(self.labels)

  def __setgreyscale__(self,mode):
    self.greyscale = mode

  def __getitem__(self, index):
      path = self.paths[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_GRAYSCALE if self.greyscale else cv2.IMREAD_COLOR)

      if self.greyscale:
        image = Image.fromarray(image, mode="L")
      else:
        image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

      image = self.transform(image)

      return image, label

In [ ]:
def calcula_mitjana_desviacio(img_files_path):
  mitjana = np.array([0.0, 0.0, 0.0], dtype=np.float64)
  desviacio = np.array([0.0, 0.0, 0.0], dtype=np.float64)

  x,y = 256,256
  pixels_totals_canal = len(img_files_path) * x * y

  for i in range(len(img_files_path)):
    imatge = cv2.imread(img_files_path[i])
    imatge = cv2.resize(imatge,(x,y))

    canal_b, canal_g, canal_r = cv2.split(imatge)

    mitjana += np.array([canal_r.sum(), canal_g.sum(), canal_b.sum()])
    #desviacio += np.array([(canal_r**2).sum(), (canal_g**2).sum(), (canal_b**2).sum()])

  mitjana = mitjana / pixels_totals_canal

  for i in range(len(img_files_path)):
    imatge = cv2.imread(img_files_path[i])
    imatge = cv2.resize(imatge,(x,y))

    canal_b, canal_g, canal_r = cv2.split(imatge)
    desviacio += np.array([((canal_r-mitjana[0])**2).sum(), ((canal_g-mitjana[1])**2).sum(), ((canal_b-mitjana[2])**2).sum()])


  desviacio = np.sqrt(desviacio / pixels_totals_canal)

  return mitjana,desviacio

In [ ]:
illnes_dictionary = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses and intraepithelial carcinoma / Bowens disease',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

img_files_1 = sorted(glob('/content/sample_data/HAM10000_images_part_1/*'))
img_files_2 = sorted(glob('/content/sample_data/HAM10000_images_part_2/*'))
img_files = img_files_1 + img_files_2

img_files = np.array(img_files)

#mitjana,desviacio = calcula_mitjana_desviacio(img_files)
#S'ha descobert amb aquesta funció que la mitjana = [194.57463374 139.13953272 145.36132088]
#I desviació= [35.92275236 38.90347617 43.33101831]
mean = torch.tensor([194.57463374, 139.13953272, 145.36132088]) /255 #rgb
std = torch.tensor([35.92275236, 38.90347617, 43.33101831]) / 255

transform = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std)

])

transform_training = transforms.Compose([
    transforms.Resize((SIZE,SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomRotation(degrees=20),
    #transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean = mean, std=std),
])

imgid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in img_files}

metadates['path'] = metadates['image_id'].map(imgid_path_dict.get)
metadates['illness'] = metadates['dx'].map(illnes_dictionary.get)
metadates['illness_code'] = metadates['dx'].map({'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6})

#Aquest illness_code s'utilitzarà com a label de la enfermetat

img_number = len(img_files)


X = metadates.drop('illness_code',axis= 1)
y = metadates['illness_code']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TESTING, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VALIDATION/(TRAINING+VALIDATION), random_state=42, stratify=y_train)

#Una bona idea seria aplicar data augmentation al conjunt d'entrenament ja que les classes estan molt desbalancejades
#Una altra bona idea seria emplear una funció de pèrdua que tengui en compte les classes desbalancejades.

#Antes de guardar els datasets en classes Formes, hauriem de caluclar la mitjana i desviació típica de les imatges
train_data = Formes(X_train['path'].to_numpy(),y_train.to_numpy(),transform)
test_data = Formes(X_test['path'].to_numpy(),y_test.to_numpy(),transform)
validation_data = Formes(X_val['path'].to_numpy(),y_val.to_numpy(),transform_training)


#prova1: [0.12,0.12,0.06,0.04,0.02,0.02]
#prova2: [0.12,0.12,0.06,0.04,0.02,0.015]
#prova3: [0.13,0.13,0.07,0.05,0.02,0.015]

print(train_data.__getdist__())
train_data.__redistribute__([0.15,0.12,0.09,0.07,0.02,0.015])
print(train_data.__getdist__())
print(train_data.len)




0    4291
1     712
2     703
3     329
4     209
5      91
6      74
Name: count, dtype: int64
0    4291
1    1188
2     949
3     711
4     552
5     156
6     116
Name: count, dtype: int64
7963


In [ ]:
def veure_imatges(train_data,std,mean):
  for i in range(len(train_data)):
    imatge,label = train_data.__getitem__(i)

    imatge = imatge * (std[:, None, None]*255) + (mean[:, None, None]*255)

    # Convert the tensor back to a NumPy array
    img_numpy = imatge.permute(1, 2, 0).numpy()  # Change from (C, H, W) to (H, W, C)
    cv2_imshow(img_numpy)
    time.sleep(5)
    clear_output(wait=True)

In [ ]:
#veure_imatges(train_data,std,mean)

Ara ja tenim el training preparat. El provarem amb el validation a continuació

##ENTRENAMENT

In [ ]:
def tria_model(numero_model):
  if numero_model == 0:
    alexnetbinary = models.alexnet(weights=None)

    alexnetbinary.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 1)
    )
    loss_fn = nn.BCEWithLogitsLoss()

    return alexnetbinary,loss_fn
  elif numero_model == 1:
    alexnetmulticlass = models.alexnet(weights=None)

    alexnetmulticlass.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 7),  # Ja que tenim 7 classes.
    nn.Softmax(dim=1)
    )
    loss_fn = nn.CrossEntropyLoss()

    return alexnetmulticlass,loss_fn

  elif numero_model == 2:
    vggbinary = models.vgg19(weights=None)
    vggbinary.classifier[6] = nn.Linear(in_features=4096, out_features=1)
    loss_fn = nn.BCEWithLogitsLoss()

    return vggbinary,loss_fn




In [ ]:
BATCH_SIZE = 64
EPOCHS = 10

train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(validation_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:


model,loss_fn = tria_model(2)

projectes = ['caltech101-Alexnet','caltech101-Vgg','caltech101-ResNet','caltech101-unet','caltech101-Propi']
architecrures_inv = {0: 'alexnet',1:'vgg',2:'resnet',3:'unet',4:'propi',5:'yolo',6:'rcnn'}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

if WEIGHTSANDBIASES:
    wandb.init(
            project=projectes[MODEL],
            config={
                "epochs": EPOCHS,
                "batch_size": BATCH_SIZE,
                "lr": learning_rate,
                "trsize":training.__len__(),
                "trdist":TRAINING,
                "vsize":validation.__len__(),
                "vdist":VAL,
                "duplicated":DUPLICAR_DADES,
                "dropout": 0.1
                })



In [ ]:
def converteix_a_binari(target):
  #{'nv': 0, 'mel': 1, 'bkl': 2, 'bcc': 3, 'akiec': 4, 'vasc': 5, 'df': 6}
  diseases = {'0': 0,'1':1,'2': 0,'3': 1,'4':1,'5':0,'6':0}
  target = ([[str(num.item())] for num in target])

  mapped_tensor = torch.tensor([[diseases[num[0]]] for num in target])
  return mapped_tensor

In [ ]:
def train(model,loss_fn,dataloader,optimizer,epoch):

  train_acc = 0
  train_f1 = 0
  train_recall = 0
  train_precision = 0
  train_loss = 0

  for batch_num, (input_img, target) in tqdm(enumerate(dataloader), desc=f"Batches (Època {epoch + 1})"):
        model.train()
        optimizer.zero_grad()

        target = torch.unsqueeze(target, 1) #separa [1,2,3,4] en [[1],[2],[3]]
        target = converteix_a_binari(target) #només si volem saber si la enfermetat es benigna o no
        target = target.to(device)


        input_img = input_img.to(device)
        output = model(input_img.to(device))

        loss = loss_fn(output, target.float())

        output = torch.sigmoid(output)

        output = (output.cpu().detach().numpy() > 0.5).astype(int)

        target = target.cpu().detach().numpy()

        model.zero_grad()
        loss.backward()

        with torch.no_grad():
            optimizer.step()

        #print(f"Pèrdua entrenament batch: {batch_num} epoch: {epoch+1}  train_loss: {loss.item()}")
        train_acc += accuracy_score(target,output)
        train_f1 += f1_score(target,output,zero_division=1)
        train_recall += recall_score(target,output,zero_division=1)
        train_precision += precision_score(target,output,zero_division=1)
        train_loss += loss.item()

  return train_acc,train_f1,train_recall,train_precision,train_loss


In [ ]:
def validate(model,data_loader,loss_fn):
  val_acc = 0
  val_f1 = 0
  val_recall = 0
  val_precision = 0
  val_loss = 0

  model.eval()
  with torch.no_grad():
      for batch_num, (input_img, target) in enumerate(val_loader):
            input_img = input_img.to(device)


            target = torch.unsqueeze(target, 1)
            target = converteix_a_binari(target) #només si volem saber si la enfermetat es benigna o no
            target = target.to(device)

            output = model(input_img)
            loss = loss_fn(output, target.float())

            output = torch.sigmoid(output)
            output = (output.cpu().detach().numpy() > 0.5).astype(int)
            target = target.cpu().detach().numpy()

            val_acc  += accuracy_score(target,output)
            val_f1 += f1_score(target,output,zero_division=1)
            val_recall += recall_score(target,output,zero_division=1)
            val_precision += precision_score(target,output,zero_division=1)

            val_loss += loss.item()

  return val_acc,val_f1,val_recall,val_precision,val_loss

In [ ]:
def execute():
  t_loss = np.zeros(EPOCHS)
  v_loss = np.zeros(EPOCHS)
  acc_t = np.zeros(EPOCHS)
  acc_v = np.zeros(EPOCHS)
  f1_t = np.zeros(EPOCHS)
  f1_v = np.zeros(EPOCHS)
  recall_t = np.zeros(EPOCHS)
  recall_v = np.zeros(EPOCHS)

  pbar = tqdm(range(1, EPOCHS + 1))  # tdqm permet tenir text dinàmic

  for epoch in pbar:

      train_loss = 0
      train_acc = 0
      train_precision = 0
      train_f1 = 0
      train_recall = 0
      val_acc = 0
      val_recall = 0
      val_f1 = 0
      val_loss = 0
      val_precision = 0

      batch_num = 1

      train_acc,train_f1,train_recall,train_precision,train_loss = train(model,loss_fn,train_loader,optimizer,epoch)

      val_acc,val_f1,val_recall,val_precision,val_loss  = validate(model,loss_fn,val_loader,optimizer,epoch)

      # RESULTATS
      train_loss /= len(train_loader)
      train_acc /= len(train_loader)
      train_f1 /= len(train_loader)
      train_recall /= len(train_loader)
      train_precision /= len(train_loader)

      print(f"Pèrdua entrenament epoch: {epoch+1}  train_loss: {train_loss}")
      print(f"Accuracy train epoch: {epoch+1}  train_acc: {train_acc}")
      print(f"F1 train epoch: {epoch+1}  train_f1: {train_f1}")
      print(f"Recall train epoch: {epoch+1}  train_recall: {train_recall}")

      val_loss /= len(val_loader)
      val_acc /= len(val_loader)
      val_f1 /= len(val_loader)
      val_recall /= len(val_loader)

      print()
      print()
      print(f"Pèrdua validació epoch: {epoch+1}  val_loss: {val_loss}")
      print(f"Accuracy val epoch: {epoch+1}  val_acc: {val_acc}")
      print(f"F1 val epoch: {epoch+1}  val_f1: {val_f1}")
      print(f"Recall val epoch: {epoch+1}  val_reall: {val_recall}")

In [ ]:
execute()

In [ ]:
# from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
# from tqdm.auto import tqdm
# import pylab as pl

# t_loss = np.zeros(EPOCHS)
# v_loss = np.zeros(EPOCHS)
# acc_t = np.zeros(EPOCHS)
# acc_v = np.zeros(EPOCHS)
# f1_t = np.zeros(EPOCHS)
# f1_v = np.zeros(EPOCHS)
# recall_t = np.zeros(EPOCHS)
# recall_v = np.zeros(EPOCHS)

# pbar = tqdm(range(1, EPOCHS + 1))  # tdqm permet tenir text dinàmic

# for epoch in pbar:

#     train_loss = 0
#     train_acc = 0
#     train_f1 = 0
#     train_recall = 0
#     val_acc = 0
#     val_recall = 0
#     val_f1 = 0
#     val_loss = 0

#     batch_num = 1


#     for batch_num, (input_img, target) in tqdm(enumerate(train_loader), desc=f"Batches (Època {epoch + 1})"):
#         model.train()
#         optimizer.zero_grad()

#         target = torch.unsqueeze(target, 1) #separa [1,2,3,4] en [[1],[2],[3]]
#         target = converteix_a_binari(target) #només si volem saber si la enfermetat es benigna o no
#         target = target.to(device)


#         input_img = input_img.to(device)
#         output = model(input_img.to(device))

#         loss = loss_fn(output, target.float())

#         output = torch.sigmoid(output)

#         output = (output.cpu().detach().numpy() > 0.5).astype(int)

#         target = target.cpu().detach().numpy()
#         train_acc += accuracy_score(target,output)
#         train_f1 += f1_score(target,output,zero_division=1)
#         train_recall += recall_score(target,output,zero_division=1)

#         model.zero_grad()
#         loss.backward()

#         with torch.no_grad():
#             optimizer.step()

#         #print(f"Pèrdua entrenament batch: {batch_num} epoch: {epoch+1}  train_loss: {loss.item()}")
#         train_loss += loss.item()

#     model.eval()
#     with torch.no_grad():
#         for batch_num, (input_img, target) in enumerate(val_loader):
#             input_img = input_img.to(device)


#             target = torch.unsqueeze(target, 1)
#             target = converteix_a_binari(target) #només si volem saber si la enfermetat es benigna o no
#             target = target.to(device)

#             output = model(input_img)
#             loss = loss_fn(output, target.float())

#             output = torch.sigmoid(output)
#             output = (output.cpu().detach().numpy() > 0.5).astype(int)
#             target = target.cpu().detach().numpy()

#             val_acc  += accuracy_score(target,output)
#             val_f1 += f1_score(target,output,zero_division=1)
#             val_recall += recall_score(target,output,zero_division=1)

#             val_loss += loss.item()

#             #print(f"Pèrdua entrenament batch validacio: {batch_num} epoch: {epoch+1}  val_loss: {val_loss.item()}")

#             # RESULTATS
#     train_loss /= len(train_loader)
#     t_loss[epoch - 1] = train_loss

#     train_acc /= len(train_loader)
#     acc_t[epoch - 1] = train_acc

#     train_f1 /= len(train_loader)
#     f1_t[epoch - 1] = train_f1

#     train_recall /= len(train_loader)
#     recall_t[epoch - 1] = train_recall
#     print(f"Pèrdua entrenament epoch: {epoch+1}  train_loss: {train_loss}")
#     print(f"Accuracy train epoch: {epoch+1}  train_acc: {train_acc}")
#     print(f"F1 train epoch: {epoch+1}  train_f1: {train_f1}")
#     print(f"Recall train epoch: {epoch+1}  train_recall: {train_recall}")

#     val_loss /= len(val_loader)
#     v_loss[epoch - 1] = val_loss

#     val_acc /= len(val_loader)
#     acc_v[epoch - 1] = val_acc

#     val_f1 /= len(val_loader)
#     f1_v[epoch - 1] = val_f1

#     val_recall /= len(val_loader)
#     recall_v[epoch - 1] = val_recall

#     print()
#     print()
#     print(f"Pèrdua validació epoch: {epoch+1}  val_loss: {val_loss}")
#     print(f"Accuracy val epoch: {epoch+1}  val_acc: {val_acc}")
#     print(f"F1 val epoch: {epoch+1}  val_f1: {val_f1}")
#     print(f"Recall val epoch: {epoch+1}  val_reall: {val_recall}")



  0%|          | 0/10 [00:00<?, ?it/s]

Batches (Època 2): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 2  train_loss: 5.732170963525772
Accuracy train epoch: 2  train_acc: 0.6850046296296297
F1 train epoch: 2  train_f1: 0.011029015279742168
Recall train epoch: 2  train_recall: 0.02019047619047619


Pèrdua validació epoch: 2  val_loss: 0.5739878278512222
Accuracy val epoch: 2  val_acc: 0.7996794871794872
F1 val epoch: 2  val_f1: 0.0
Recall val epoch: 2  val_reall: 0.0


Batches (Època 3): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 3  train_loss: 0.565360979795456
Accuracy train epoch: 3  train_acc: 0.6925370370370371
F1 train epoch: 3  train_f1: 0.005761547823373381
Recall train epoch: 3  train_recall: 0.0032644352426961123


Pèrdua validació epoch: 3  val_loss: 0.5274392469571187
Accuracy val epoch: 3  val_acc: 0.8118990384615384
F1 val epoch: 3  val_f1: 0.038461538461538464
Recall val epoch: 3  val_reall: 0.038461538461538464


...

Pèrdua entrenament epoch: 9  train_loss: 0.47920003747940065
Accuracy train epoch: 9  train_acc: 0.7479722222222221
F1 train epoch: 9  train_f1: 0.4718602183160907
Recall train epoch: 9  train_recall: 0.38709020251040027


Pèrdua validació epoch: 9  val_loss: 0.6221034756073585
Accuracy val epoch: 9  val_acc: 0.6786858974358974
F1 val epoch: 9  val_f1: 0.3087914673791075
Recall val epoch: 9  val_reall: 0.40809123945436154


Batches (Època 10): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 10  train_loss: 0.46130999755859375
Accuracy train epoch: 10  train_acc: 0.7550185185185184
F1 train epoch: 10  train_f1: 0.506996416921673
Recall train epoch: 10  train_recall: 0.4346816611005045


Pèrdua validació epoch: 10  val_loss: 0.6744141120177048
Accuracy val epoch: 10  val_acc: 0.5939503205128205
F1 val epoch: 10  val_f1: 0.3469450386306955
Recall val epoch: 10  val_reall: 0.5994049725578066


Batches (Època 11): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 11  train_loss: 0.4509991974830627
Accuracy train epoch: 11  train_acc: 0.7672546296296296
F1 train epoch: 11  train_f1: 0.5271488821647131
Recall train epoch: 11  train_recall: 0.4531370739357982


Pèrdua validació epoch: 11  val_loss: 0.4937738718894812
Accuracy val epoch: 11  val_acc: 0.7852564102564104
F1 val epoch: 11  val_f1: 0.2890799062366568
Recall val epoch: 11  val_reall: 0.23472139034921838
